In [390]:
from z3 import *
class f2Poly:
    def __init__(self, terms=None):
        self.terms = {}
        if terms:
            for (i, j), coef in terms.items():
                if coef % 2 == 1:  # 模 2
                    self.terms[(i, j)] = 1  # 只存 1
    def __add__(self, other):
        result = self.terms.copy()
        for (i, j), coef in other.terms.items():
            if (i, j) in result:
                del result[(i, j)]  # 1+1=0
            else:
                result[(i, j)] = 1
        return f2Poly(result)
    def __mul__(self, other):
        if isinstance(other, int):
            if other % 2 == 0:
                return f2Poly()  # 偶数倍 → 0
            else:
                return self  # 奇数倍 → 自身（在 F2 中 1*f = f, 3*f = f）
        else:
            result = {}
            for (i1, j1) in self.terms:
                for (i2, j2) in other.terms:
                    i_new, j_new = i1 + i2, j1 + j2
                    key = (i_new, j_new)
                    if key in result:
                        del result[key]
                    else:
                        result[key] = 1
            return f2Poly(result)
    def __pow__(self, n):
        if n == 0:
            return f2Poly({(0, 0): 1})  # f^0 = 1
        if len(self.terms) == 0:
            if n >= 0:
                return f2Poly()  # 0^n = 0 (n>0)
            else:
                raise ValueError("0 的负幂次无定义")
        elif len(self.terms) == 1:
            (i, j), coef = next(iter(self.terms.items()))
            return f2Poly({(i * n, j * n): 1})
        else:
            # 多项式项数 > 1
            if n < 0:
                raise ValueError(f"多项式 {self} 不可逆，无法计算负幂次")
            elif n == 1:
                return self
            elif n % 2 == 0:
                return (self * self) ** (n // 2)
            else:
                return self * (self ** (n - 1))
    def degrees(self):
        if not self.terms:
            return (0, 0)
        i_vals = [i for (i, j) in self.terms.keys()]
        j_vals = [j for (i, j) in self.terms.keys()]
        return (max(i_vals), max(j_vals))
    def __repr__(self):
        if not self.terms:
            return "0"
        sorted_terms = sorted(self.terms.keys())
        terms = []
        for (i, j) in sorted_terms:
            xi = f"x^{i}" if i != 0 else ""
            yj = f"y^{j}" if j != 0 else ""
            term = xi + yj
            if term == "":
                term = "1"
            terms.append(term)
        return " + ".join(terms)
def antimap(poly):
    inverted_terms = {}
    for (i, j), coef in poly.terms.items():
        inverted_terms[(-i, -j)] = coef  # 系数不变（在 F2 中）
    return f2Poly(inverted_terms)
def commute(a, b):
    Lambda = [[0,0,1,0],[0,0,0,1],[1,0,0,0],[0,1,0,0]]
    result = f2Poly()
    for i in range(4):
        for j in range(4):
            if Lambda[i][j]:
                term = a[i][0].antimap() * b[j][0]
                result += term
    return result
def excimap(*A, oper):
    result = []
    for a in A:
        # 计算 commute(a, oper)
        excitation = commute(a, oper)
        result.append(excitation)
    return result  # 返回 [F2Poly, F2Poly, ...]，就是一个行向量
def tdmap(f, m):
    result = []
    for j in range(-m, m + 1):      # y 的指数从 -m 到 m
        for i in range(-m, m + 1):  # x 的指数从 -m 到 m
            shift = f2Poly({(i, j): 1})  # x^i y^j
            shifted_f = [shift * i for i in f]         # 平移后的多项式
            result.append(shifted_f)
    return result
def shape(obj):
    if not obj:
        return (0,)  # 空对象
    if isinstance(obj[0], (list, tuple)):
        # 是矩阵：obj = [ [...], [...] ]
        rows = len(obj)
        cols = len(obj[0])
        return (rows, cols)
    else:
        # 是向量：obj = [a, b, c, d]
        return (len(obj),)

def hstack(A,B):
    if len(A) != len(B):
        raise ValueError("矩阵行数必须相同")
    result = []
    for i in range(len(A)):
        row = A[i] + B[i]  # 直接拼接
        result.append(row)
    return result
def I(n):
    zero = f2Poly()
    one  = f2Poly({(0,0): 1})
    return [[one if i == j else zero for j in range(n)] for i in range(n)]
def split(augmented):
    if not augmented:
        return [], []
    m = len(augmented)
    total_cols = len(augmented[0])
    n = total_cols - m
    if n < 0:
        raise ValueError("矩阵列数小于行数，无法分割")
    left = []   # 原矩阵部分 A'
    right = []  # 单位阵部分 B'
    for row in augmented:
        left.append(row[:n])   # 前 n 列
        right.append(row[n:])  # 后 m 列
    return left, right
def mulmatrix(A, B):
    if not A or not B:
        return []
    m = len(A)
    n = len(A[0])
    if len(B) != n:
        raise ValueError(f"矩阵维度不匹配：A 是 {m}x{n}，B 是 {len(B)}x{len(B[0])}")
    p = len(B[0])
    zero = f2Poly()
    C = [[zero for _ in range(p)] for _ in range(m)]
    for i in range(m):
        for j in range(p):
            total = f2Poly()  # 零多项式，累加用
            for k in range(n):
                prod = A[i][k] * B[k][j]  # f2Poly 乘法
                total = total + prod      # f2Poly 加法（F2 上）
            C[i][j] = total
    return C
def t(matrix):
    if not matrix or not matrix[0]:
        return []
    return [list(row) for row in zip(*matrix)]





x1=[[f2Poly({(0,0):1})],
    [f2Poly()],
    [f2Poly()],
    [f2Poly()]]
x2=[[f2Poly()],
    [f2Poly({(0,0):1})],
    [f2Poly()],
    [f2Poly()]]
z1=[[f2Poly()],
    [f2Poly()],
    [f2Poly({(0,0):1})],
    [f2Poly()]]
z2=[[f2Poly()],
    [f2Poly()],
    [f2Poly()],
    [f2Poly({(0,0):1})]]
# 示例 1: 3x + 2y → 模 2 后是 x
s1=[
    [f2Poly({(0, 0): 1,(-1,0):1})],
    [f2Poly({(0, 0): 1,(0,-1):1})],
    [f2Poly()],
    [f2Poly()]
]
s2=[
    [f2Poly()],
    [f2Poly()],
    [f2Poly({(0, 0): 1,(0,1):1})],
    [f2Poly({(0, 0): 1,(1,0):1})],
]
m=11
m1=[]

def has_zero_row(matrix):
    """
    检查矩阵中是否有全零行
    返回: bool
    """
    if not matrix:
        return False

    zero = f2Poly()  # 或者直接用 len(p.terms) == 0 判断
    for i, row in enumerate(matrix):
        # 判断该行是否全为零
        if all(len(p.terms) == 0 for p in row):
            return True  # 找到全零行
    return False
print(f2Poly.degrees(f2Poly({(-1,-1):1})))

(-1, -1)


In [383]:
# 定义基本变量
x = f2Poly({(1, 0): 1})
y = f2Poly({(0, 1): 1})
one = f2Poly({(0, 0): 1})
zero = f2Poly()

matrix = [[x+y,x**2+y,x+y**2],
          [x,y,zero],
          [x**2,x**3+y**2+x*y,x**2+y**2]]
print(matrix)
print(gaussian_elimination(matrix))

[[y^1 + x^1, y^1 + x^2, y^2 + x^1], [x^1, y^1, 0], [x^2, y^2 + x^1y^1 + x^3, y^2 + x^2]]
[[x^1, y^1, 0], [y^1 + x^1, x^2, y^2 + x^1], [x^1 + x^2, y^2 + x^1y^1 + x^3, y^2 + x^2]]


In [396]:
# f2xy_elim_robust.py
# 修正版：更健壮的拷贝/输入处理。入口：eliminate_polynomial_matrix(M, verbose=False)

from copy import deepcopy

class f2Poly:
    def __init__(self, terms=None):
        self.terms = {}
        if terms:
            if isinstance(terms, dict):
                for (i,j), coef in terms.items():
                    if int(coef) % 2 == 1:
                        self.terms[(int(i), int(j))] = 1
            else:
                # iterable of exponent pairs
                for (i,j) in terms:
                    self.terms[(int(i), int(j))] = 1

    # 兼容性：如果某个老版本没有 copy，我们仍然提供
    def copy(self):
        return f2Poly(dict(self.terms))

    def is_zero(self):
        return len(self.terms) == 0

    def __add__(self, other):
        if not isinstance(other, f2Poly):
            raise TypeError("加法要求 f2Poly")
        res = dict(self.terms)
        for (i,j) in other.terms:
            if (i,j) in res:
                del res[(i,j)]
            else:
                res[(i,j)] = 1
        return f2Poly(res)

    def __radd__(self, other):
        if other == 0:
            return self.copy()
        return self.__add__(other)

    def __mul__(self, other):
        if isinstance(other, tuple) and len(other) == 2:
            a,b = other
            res = {}
            for (i,j) in self.terms:
                key = (i + int(a), j + int(b))
                if key in res:
                    del res[key]
                else:
                    res[key] = 1
            return f2Poly(res)
        elif isinstance(other, f2Poly):
            res = {}
            for (i1,j1) in self.terms:
                for (i2,j2) in other.terms:
                    key = (i1+i2, j1+j2)
                    if key in res:
                        del res[key]
                    else:
                        res[key] = 1
            return f2Poly(res)
        elif isinstance(other, int):
            return self.copy() if other % 2 == 1 else f2Poly()
        else:
            raise TypeError("不支持与类型 {} 相乘".format(type(other)))

    def multiply_by_monomial(self, a, b):
        return self * (int(a), int(b))

    def swap_xy(self):
        return f2Poly({(j,i):1 for (i,j) in self.terms})

    def leading_y(self):
        if self.is_zero():
            return None
        return max(j for (i,j) in self.terms.keys())

    def leading_x(self):
        if self.is_zero():
            return None
        return max(i for (i,j) in self.terms.keys())

    def pure_y_monomial(self):
        if len(self.terms) != 1:
            return False
        (i,j) = next(iter(self.terms.keys()))
        return i == 0

    def pure_x_monomial(self):
        if len(self.terms) != 1:
            return False
        (i,j) = next(iter(self.terms.keys()))
        return j == 0

    def degrees(self):
        if self.is_zero():
            return (0,0)
        i_vals = [i for (i,j) in self.terms.keys()]
        j_vals = [j for (i,j) in self.terms.keys()]
        return (max(i_vals), max(j_vals))

    def __repr__(self):
        if self.is_zero():
            return "0"
        items = []
        for (i,j) in sorted(self.terms.keys()):
            xi = (f"x^{i}" if i != 0 else "")
            yj = (f"y^{j}" if j != 0 else "")
            s = xi + yj
            if s == "":
                s = "1"
            items.append(s)
        return " + ".join(items)

# ---------------- 工具：把任意输入转换为合适的 f2Poly 短拷贝 ----------------
def _to_f2poly(obj):
    """
    尝试把 obj 转换/拷贝为 f2Poly 实例：
      - 若有 copy() 则用之（保留原类型的 copy 行为）
      - 否则若有 .terms 属性且是 dict，则用 f2Poly(obj.terms)
      - 否则尝试用 f2Poly(obj) 构造（支持 dict 或 iterable of pairs）
    """
    if isinstance(obj, f2Poly):
        # 使用其 copy 方法（若存在）
        try:
            return obj.copy()
        except Exception:
            return f2Poly(dict(getattr(obj, "terms", {})))
    # 若传入的是 dict（例如 {(i,j):coef}）
    if isinstance(obj, dict):
        return f2Poly(obj)
    # 若有 terms 属性
    if hasattr(obj, "terms") and isinstance(getattr(obj, "terms"), dict):
        return f2Poly(dict(getattr(obj, "terms")))
    # 尝试直接构造
    try:
        return f2Poly(obj)
    except Exception:
        # 最后兜底：若无法转，则返回零多项式（或抛错）
        raise TypeError("无法将对象转换为 f2Poly：{}".format(type(obj)))

def _ensure_matrix2d(M):
    """
    将输入 M 规范化为 2D 列表，元素为 f2Poly（浅拷贝）：
      - 若 M 是二维 list-of-lists（假设每项为多项式对象），拷贝其元素；
      - 若 M 是一维 list（元素为多项式），视为 1 x n（单行）；
      - 若 M 是单个多项式对象，视为 1 x 1。
    返回规范化后的二维列表（元素为 f2Poly）。
    """
    # 如果是 list of lists（2D）
    if isinstance(M, list) and M and isinstance(M[0], list):
        nr = len(M); nc = len(M[0])
        A = [[_to_f2poly(M[r][c]) for c in range(nc)] for r in range(nr)]
        return A
    # 如果是一维 list（当作 1 x n）
    if isinstance(M, list):
        row = [_to_f2poly(x) for x in M]
        return [row]
    # 其他对象（当作单个多项式）
    return [[_to_f2poly(M)]]

# ---------------- 基本矩阵操作（基于 f2Poly） ----------------
def zeros_matrix(nr, nc):
    return [[f2Poly() for _ in range(nc)] for __ in range(nr)]

def copy_matrix_from_normalized(A):
    # A 已经是二维 list，元素为 f2Poly；返回深拷贝
    return [[_to_f2poly(A[r][c]) for c in range(len(A[0]))] for r in range(len(A))]

def add_rows(M, target_row, source_row, monomial_shift=(0,0)):
    a,b = monomial_shift
    nc = len(M[0])
    for c in range(nc):
        M[target_row][c] = M[target_row][c] + M[source_row][c].multiply_by_monomial(a, b)

def transpose_matrix(M):
    nr = len(M); nc = len(M[0])
    T = zeros_matrix(nc, nr)
    for r in range(nr):
        for c in range(nc):
            T[c][r] = M[r][c].swap_xy()
    return T

# ---------------- 消元逻辑 ----------------
def eliminate_y_columnwise(M, verbose=False, max_iter_each_col=200):
    nr = len(M); nc = len(M[0])
    for col in range(nc):
        it = 0
        while True:
            it += 1
            rows = [r for r in range(nr) if not M[r][col].is_zero()]
            if len(rows) <= 1:
                break
            degs = {r: M[r][col].leading_y() for r in rows}
            pivot = min(rows, key=lambda r: degs[r] if degs[r] is not None else float('inf'))
            pivot_deg = degs[pivot]
            if verbose:
                print(f"[col {col} iter {it}] rows={rows}, pivot={pivot}, pivot_deg={pivot_deg}")
            changed = False
            for r in rows:
                if r == pivot:
                    continue
                deg_r = degs[r]
                if deg_r is None or pivot_deg is None:
                    continue
                if deg_r >= pivot_deg:
                    shift = deg_r - pivot_deg
                    add_rows(M, r, pivot, (0, shift))
                    changed = True
            if not changed:
                break
            if it > max_iter_each_col:
                if verbose:
                    print("达到迭代上限，退出该列")
                break
    return M

def eliminate_by_swapping_xy_and_repeat(M, verbose=False):
    eliminate_y_columnwise(M, verbose=verbose)
    T = transpose_matrix(M)
    eliminate_y_columnwise(T, verbose=verbose)
    M2 = transpose_matrix(T)
    nr = len(M); nc = len(M[0])
    for r in range(nr):
        for c in range(nc):
            M[r][c] = M2[r][c]
    return M

def final_merge_and_row_move(M, verbose=False):
    nr = len(M); nc = len(M[0])
    for col in range(nc):
        rows = [r for r in range(nr) if not M[r][col].is_zero()]
        if not rows:
            continue
        if len(rows) == 1:
            r = rows[0]
            if col < nr and r != col:
                M[r], M[col] = M[col], M[r]
                if verbose:
                    print(f"swap rows {r} <-> {col}")
            continue
        pure_x = [r for r in rows if M[r][col].pure_x_monomial()]
        pure_y = [r for r in rows if M[r][col].pure_y_monomial()]
        if pure_x and pure_y:
            rx = pure_x[0]
            ry = pure_y[0]
            ax = M[rx][col].leading_x()
            by = M[ry][col].leading_y()
            for c in range(nc):
                M[rx][c] = M[rx][c].multiply_by_monomial(0, by)
                M[ry][c] = M[ry][c].multiply_by_monomial(ax, 0)
            add_rows(M, ry, rx, (0,0))
            if all(M[ry][c].is_zero() for c in range(nc)):
                if col < nr and rx != col:
                    M[rx], M[col] = M[col], M[rx]
            else:
                nonzero = rx if any(not M[rx][c].is_zero() for c in range(nc)) else ry
                if col < nr and nonzero != col:
                    M[nonzero], M[col] = M[col], M[nonzero]
            if verbose:
                print(f"merged pure-x row {rx} and pure-y row {ry} in column {col}")
        else:
            r0 = rows[0]
            if col < nr and r0 != col:
                M[r0], M[col] = M[col], M[r0]
                if verbose:
                    print(f"fallback move row {r0} -> {col}")
    return M

# ---------------- 入口函数（健壮版） ----------------
def eliminate_polynomial_matrix(M, verbose=False):
    """
    入口：接受多种输入格式（单个 f2Poly、1D list、2D list），返回消元后的新矩阵（2D list of f2Poly）。
    """
    norm = _ensure_matrix2d(M)          # 规范化并转换为 f2Poly 元素
    A = copy_matrix_from_normalized(norm)  # 深拷贝为工作矩阵
    eliminate_by_swapping_xy_and_repeat(A, verbose=verbose)
    final_merge_and_row_move(A, verbose=verbose)
    return A
print(eliminate_polynomial_matrix(matrix,verbose=False))

[[y^1, 0, y^1 + x^1], [y^1 + x^1 + x^1y^2 + x^2y^2, y^1 + x^1y^1 + x^1y^2 + x^2 + x^2y^2 + x^3y^2, y^2 + x^1y^2], [y^1 + x^1y^1 + x^1y^2 + x^2 + x^2y^2, x^1y^1 + x^1y^2 + x^2y^1 + x^2y^2 + x^3 + x^3y^2, y^1 + y^2 + x^1 + x^1y^2]]
